<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-PyTorch/blob/main/Gradio_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Build an App with Gradio?

After training a machine learning model, the next step is to show it to others by creating a demo. One way to do this is by using Gradio, and hosting it on HuggingFace Spaces. This is a quick and easy process, taking less than 10 minutes to deploy a model. In this example, we will be using a classic CNN pet classifier.

# Training a pet classifier

Before creating a demo, we must first train a model. Let's train a simple ResNet50 pet classifier on the Oxford Pets dataset using fastai.

In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.PETS)
dls = ImageDataLoaders.from_name_re(path, get_image_files(path/'images'), pat='(.+)_\d+.jpg', item_tfms=Resize(460), batch_tfms=aug_transforms(size=224, min_scale=0.75))
learn = vision_learner(dls, models.resnet50, metrics=accuracy)
learn.fine_tune(1)
learn.path = Path('.')
learn.export()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.920208,0.314169,0.905954,01:19


epoch,train_loss,valid_loss,accuracy,time
0,0.443065,0.266486,0.917456,01:27


# Building a demo web app with Gradio

First let's load our model:

In [3]:
learn = load_learner('export.pkl')

Next, let's create a prediction function our model:

In [4]:
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

Let's create an interface and launch it using Gradio.

In [ ]:
#!pip install gradio

In [7]:
import gradio as gr
gr.Interface(fn=predict, 
             inputs=gr.inputs.Image(shape=(512, 512)), 
             outputs=gr.outputs.Label(num_top_classes=3)).launch(share=True)

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e3de5391-e452-437d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Awesome! We created an app with Gradio.

# Customizing the Gradio app

Gradio offers a variety of options for customizing our application. First, we have the ability to provide a title and description for our application, which will be displayed at the top, preceding the input and output components.

In [8]:
title = "Pet Breed Classifier"
description = "A demonstration of a pet breed classifier utilizing the Oxford Pets dataset and fastai library, specifically designed for use with Gradio and HuggingFace Spaces."

Additionally, we have the option to include a link at the bottom of our demonstration. In this case, I will provide a link to the following blog post.

In [9]:
article="<p style='text-align: center'><a href='https://github.com/TirendazAcademy' target='_blank'>Repo</a></p>"

Another feature is the ability to supply example inputs for users to test the application.

In [10]:
examples = ['cat.jpg']

Let's put it all into one file which we name app.py:

In [12]:
import gradio as gr
from fastai.vision.all import *
import skimage

learn = load_learner('export.pkl')

labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

title = "FastAI Pet Breed Classifier"
description = "A demonstration of a pet breed classifier utilizing the Oxford Pets dataset and fastai library, specifically designed for use with Gradio and HuggingFace Spaces."
article="<p style='text-align: center'><a href='https://github.com/TirendazAcademy' target='_blank'>GitHub Repo</a></p>"

gr.Interface(fn=predict,
             inputs=gr.inputs.Image(shape=(512, 512)),
             outputs=gr.outputs.Label(num_top_classes=3),
             title=title,
             description=description,
             article=article,
             examples=['cat.jpg']).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

It is also recommended to create a requirements.txt file, which will enable us to install the necessary packages in any environment we use.

```
fastai
scikit-image
```

After that you can perform as following steps:
- Create a space in HuggingFace named pet-classifier
- Clone the repository to a local directory:
`git clone https://huggingface.co/spaces/tirendaz/pet-classifier`
- Add the app.py, requirements.txt, export.pkl, and cat.jpg files.
- Our model file export.pkl is too big to be handled by git. So instead we need to use git-lfs.
```
git lfs install
git lfs track "*.pkl"
git commit -m "update .gitattributes so git lfs will track .pkl files"
git add .
git commit -m "Added files"
git push
```

Congratulations! In a short period of time, you have successfully trained a pet classification model using fastai, created a demo interface with Gradio, and made it available to the public for free through a HuggingFace Space.



- Thanks for reading.
- Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) |[GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎

## Resources:

- [Gradio + HuggingFace Spaces: A Tutorial](https://tmabraham.github.io/blog/gradio_hf_spaces_tutorial)